In [5]:
import pandas as pd

# =========================================================
# 1. Load driving log data
# =========================================================
df = pd.read_csv("data/driving_log.csv")

# =========================================================
# 2. Threshold definitions (m/s^2)
# =========================================================
HARD_BRAKE_THRESHOLD = -3.0
HARD_ACCEL_THRESHOLD = 3.0

# =========================================================
# 3. Detect hard driving events
# =========================================================
hard_brake_df = df[df["accel_mps2"] < HARD_BRAKE_THRESHOLD]
hard_accel_df = df[df["accel_mps2"] > HARD_ACCEL_THRESHOLD]

hard_brake_count = len(hard_brake_df)
hard_accel_count = len(hard_accel_df)
total_hard_events = hard_brake_count + hard_accel_count

# =========================================================
# 4. Classification by total event count
# =========================================================
def classify_driver_behavior_by_count(total_events):
    if total_events <= 2:
        return "Smooth Driver"
    elif total_events <= 5:
        return "Normal Driver"
    else:
        return "Aggressive Driver"

driver_type_count = classify_driver_behavior_by_count(total_hard_events)

# =========================================================
# 5. Time-based normalization (events per minute)
# =========================================================
total_time_sec = df["time_sec"].max() - df["time_sec"].min()

MIN_TIME_SEC = 60  # minimum valid duration (1 min)

if total_time_sec >= MIN_TIME_SEC:
    events_per_min = total_hard_events / (total_time_sec / 60)
else:
    events_per_min = None

# =========================================================
# 6. Classification by event rate
# =========================================================
def classify_driver_behavior_by_rate(events_per_min):
    if events_per_min is None:
        return "Insufficient Data"
    elif events_per_min < 3:
        return "Smooth Driver"
    elif events_per_min < 6:
        return "Normal Driver"
    else:
        return "Aggressive Driver"

driver_type_rate = classify_driver_behavior_by_rate(events_per_min)

# =========================================================
# 7. Distance-based normalization (events per km)
# =========================================================
avg_speed_kmh = df["speed_kmh"].mean()
distance_km = avg_speed_kmh * (total_time_sec / 3600)

if distance_km > 0:
    events_per_km = total_hard_events / distance_km
else:
    events_per_km = None

# =========================================================
# 8. Final Result Output
# =========================================================
print("Driver Behavior Analysis Result")
print("--------------------------------")
print(f"Hard Acceleration Events : {hard_accel_count}")
print(f"Hard Braking Events      : {hard_brake_count}")
print(f"Total Hard Events        : {total_hard_events}")
print(f"Total Time (sec)         : {total_time_sec:.1f}")

if events_per_min is not None:
    print(f"Events per Minute        : {events_per_min:.2f}")
else:
    print("Events per Minute        : Insufficient Data")

if events_per_km is not None:
    print(f"Events per km            : {events_per_km:.2f}")
else:
    print("Events per km            : Insufficient Data")

print("--------------------------------")
print(f"Classification (Count)   : {driver_type_count}")
print(f"Classification (Rate)    : {driver_type_rate}")



Driver Behavior Analysis Result
--------------------------------
Hard Acceleration Events : 3
Hard Braking Events      : 3
Total Hard Events        : 6
Total Time (sec)         : 9.0
Events per Minute        : Insufficient Data
Events per km            : 94.12
--------------------------------
Classification (Count)   : Aggressive Driver
Classification (Rate)    : Insufficient Data
